In [ ]:
# Configuration - Set your catalog and schema
catalog = "main"
schema = "jpg"
table_name = f"{catalog}.{schema}.customers"
filter_function = f"{catalog}.{schema}.simple_filter"
country_filter_function = f"{catalog}.{schema}.country_row_filter"
mask_function = f"{catalog}.{schema}.mask_email"
mapping_table = f"{catalog}.{schema}.map_rls_country"

print(f"Using catalog: {catalog}")
print(f"Using schema: {schema}")
print(f"Target table: {table_name}")


In [ ]:
# Query the customers table (before applying any filters)
spark.sql(f"SELECT * FROM {table_name}").display()

In [ ]:
# Create a simple row filter function
spark.sql(f"""
CREATE OR REPLACE FUNCTION {filter_function}(country STRING)
RETURN 
  is_account_group_member('tuanis_admins')
  OR
  country IN ('Brazil', 'United States')
""")

In [ ]:
# Apply the row filter to the table
spark.sql(f"""
ALTER TABLE {table_name} 
SET ROW FILTER {filter_function} ON (country)
""")


In [ ]:
# Remove the row filter from the table
spark.sql(f"""
ALTER TABLE {table_name} 
DROP ROW FILTER
""")


In [ ]:
# Create a mapping table for row-level security
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {mapping_table} (
  group STRING,
  countries ARRAY<STRING>
)
""")

In [ ]:
# Insert sample data into the mapping table
spark.sql(f"""
INSERT INTO {mapping_table} (group, countries) VALUES
  ('northamer', ARRAY('USA', 'CANADA', 'MEXICO')),
  ('europe', ARRAY('FR', 'SPAIN'))
""")

In [ ]:
# Create an advanced row filter function using the mapping table
spark.sql(f"""
CREATE OR REPLACE FUNCTION {country_filter_function}(country STRING)
RETURN 
  is_account_group_member('tuanis_admins')
  OR
  EXISTS (
    SELECT 1
    FROM {mapping_table} m
    WHERE 
    array_contains(m.countries, country)
    AND is_account_group_member(m.group)
  )
""")


In [ ]:
# Create a column masking function for email addresses
spark.sql(f"""
CREATE OR REPLACE FUNCTION {mask_function}(email STRING)
RETURN
  IF(
    is_account_group_member('tuanis_admins'),
    email,
    CONCAT('***@', SPLIT(email, '@')[1])
  )
""")


In [ ]:
# Apply column masking to the email column
spark.sql(f"""
ALTER TABLE {table_name}
ALTER COLUMN email SET MASK {mask_function}
""")


In [ ]:
# Remove column masking from the email column
spark.sql(f"""
ALTER TABLE {table_name}
ALTER COLUMN email DROP MASK
""")
